**Current working directory: ** /data/home/group8/azure_notebooks

https://notebooks.azure.com/help/jupyter-notebooks/ help link

In [ ]:
!pip install seaborn

In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy import interp
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import math
import scipy.stats as ss
import matplotlib.pyplot as plt
from collections import Counter

%matplotlib inline


In [2]:
temp= !pwd
working_dir=temp[0]

In [ ]:
crash_df = pd.read_csv(working_dir+'/Crashes_Last_Five_Years.csv')

crash_df.shape

Use http://data.vicroads.vic.gov.au/metadata/Crashes_Last_Five_Years%20-%20Open%20Data.html to know what each variable means

In [ ]:
for i,variable in enumerate(crash_df.columns):
    print(i+1,variable)

<b>Alcohol Related Crashes BAC>0.001 and road user type=driver,rider,cyclist,pedestrian</b>

In [ ]:
#IDEMPOTENT
alcoholic_crashes_df = crash_df[crash_df['ALCOHOL_RELATED']=='Yes']
alcoholic_crashes_df.shape#(2479,65)
alcoholic_crashes_df.drop(['ALCOHOL_RELATED'],axis=1,inplace=True)
alcoholic_crashes_df.shape#(2479,64)

In [ ]:
# sns.pairplot(alcoholic_crashes_df)
sns.pairplot(alcoholic_crashes_df, vars = list(numerical_fields), hue = 'DAY_OF_WEEK', diag_kind = 'kde', plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},size = 4)


## 2014 to 2019

In [ ]:
df_2014_to_2019_cleaned = pd.read_csv(working_dir+'/2014_to_2019_cleaned.csv')
df_2014_to_2019_cleaned.drop(columns=['Unnamed: 0'],axis=1,inplace=True)


### Plotting individually

In [ ]:
df=df_2014_to_2019_cleaned
plot_scatter = True
numerical_fields = df._get_numeric_data().columns
categorical_fields = list(set(df.columns)-set(numerical_fields))
assert len(categorical_fields)+len(numerical_fields)==len(df.columns)
for i in numerical_fields:
    for j in numerical_fields:
        if i==j:
            sns.distplot(df[i])
            plt.show()
        elif i!=j and plot_scatter:
            sns.scatterplot(x=i,y=j,data=df)
            plt.show()

### Collision data v1 analysis

Reading the file

In [139]:
my_data = pd.read_csv(working_dir+'/collision_data_1.csv')
my_data.shape

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(314349, 24)

Selecting the records where  <b>alcohol was consumed<b>

In [140]:
my_data_df_1 = my_data[my_data['alcohol_involvement_flag']=='Y']
my_data_df_1.drop(['alcohol_involvement_flag'],axis=1,inplace=True)
my_data_df_1.shape

/home/nizam/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(13721, 23)

In [141]:
df=my_data_df_1
del my_data_df_1
del my_data
df.shape

(13721, 23)

<b>Formatting the date and coordinates<b>

In [142]:
df['collision_date']= pd.to_datetime(df['collision_date'])  #formating date
df[['year','month','date']] = df.collision_date.apply(lambda x: pd.Series(x.strftime("%Y,%m,%d").split(",")))
df.drop(['collision_date'],axis=1,inplace=True)
df = df[(df['speed_zone']<200)]
df.drop(df[df.latitude =='.'].index,inplace=True)
df.drop(df[df.longitude =='.'].index,inplace=True)
df['latitude']= pd.to_numeric(df['latitude'])  #formating latitude and longitude
df['longitude']= pd.to_numeric(df['longitude']) 
df.shape

(12448, 25)

Filtering records yearwise

In [136]:
df_2009 = df[(df['year']=='2009')]
df_2010 = df[(df['year']=='2010')]
df_2011 = df[(df['year']=='2011')]
df_2012 = df[(df['year']=='2012')]
df_2013 = df[(df['year']=='2013')]
df_2014 = df[(df['year']=='2014')]
df_2015 = df[(df['year']=='2015')]
df_2016 = df[(df['year']=='2016')]
df_2017 = df[(df['year']=='2017')]
df_2018 = df[(df['year']=='2018')] 
print("2009",df_2009.shape)
print("2010",df_2010.shape)
print("2011",df_2011.shape)
print("2012",df_2012.shape)
print("2013",df_2013.shape)
print("2014",df_2014.shape)
print("2015",df_2015.shape)
print("2016",df_2016.shape)
print("2017",df_2017.shape)
print("2018",df_2018.shape)

2009 (2401, 25)
2010 (1981, 25)
2011 (1615, 25)
2012 (1165, 25)
2013 (981, 25)
2014 (931, 25)
2015 (832, 25)
2016 (884, 25)
2017 (872, 25)
2018 (786, 25)


In [143]:
df_2009

,tis_number,collision_time,latitude,longitude,street_number,street_name,street_type,suburb,postcode,lga_name,...,dca_code,dca_code_description,speed_zone,motorcyclist,seat_belt_not_worn,helmet_not_worn,heavy_vehicle,year,month,day
1,T20090000002,5,-37.884967,145.006103,.,GLEN HUNTLY,RD,ELSTERNWICK,3185.0,GLEN EIRA,...,181,Off Path on Curve - off Right Bend into Object...,60.0,NaN,NaN,NaN,NaN,2009,01,01
6,T20090000010,245,-37.770396,144.795132,.,WESTERN,HWY,DEER PARK,3023.0,BRIMBANK,...,110,Vehicles / Adjacent Dirs (Inters) - Cross Traffic,80.0,NaN,NaN,NaN,NaN,2009,01,01
10,T20090000016,305,-36.794980,144.336200,.,ALEXANDER,CL,STRATHFIELDSAYE,3551.0,GREATER BENDIGO,...,160,On Path – Parked,50.0,NaN,NaN,NaN,NaN,2009,01,01
12,T20090000018,245,-37.783066,145.165642,22,BELLEVUE,AVE,DONCASTER EAST,3109.0,MANNINGHAM,...,171,Off Path on Straight - Left off Carriageway in...,50.0,NaN,NaN,NaN,NaN,2009,01,01
17,T20090000026,720,-37.961952,145.018981,.,BLUFF,RD,BLACK ROCK,3193.0,BAYSIDE,...,172,Off Path on Straight - off Carriageway to Right,60.0,NaN,NaN,NaN,NaN,2009,01,01
19,T20090000031,750,-37.064377,142.784795,83,NAPIER,ST,STAWELL,3380.0,NORTHERN GRAMPIANS,...,160,On Path – Parked,60.0,NaN,NaN,NaN,NaN,2009,01,01
21,T20090000034,630,-37.801130,144.406115,.,BACCHUS MARSH,RD,BALLIANG EAST,3340.0,GREATER GEELONG,...,151,Overtaking - out of Control,100.0,NaN,NaN,NaN,NaN,2009,01,01
24,T20090000039,935,-37.735610,142.113520,.,GLENELG,HWY,HAMILTON,3300.0,SOUTHERN GRAMPIANS,...,172,Off Path on Straight - off Carriageway to Right,100.0,NaN,NaN,NaN,NaN,2009,01,01
25,T20090000043,515,-37.979876,147.055164,.,STRATFORD-MAFFRA,RD,STRATFORD,3862.0,WELLINGTON,...,172,Off Path on Straight - off Carriageway to Right,100.0,NaN,seat_belt_not_worn,NaN,NaN,2009,01,01
28,T20090000046,15,-36.368365,142.986551,.,WALKER,ST,DONALD,3480.0,BULOKE,...,170,Off Path on Straight - off Carriageway to Left,40.0,NaN,NaN,NaN,NaN,2009,01,01


In [137]:
# df_2009 = df[(df['collision_date']>='2009-01-01') & (df['collision_date']<='2009-12-31')]
# df_2010 = df[(df['collision_date']>='2010-01-01') & (df['collision_date']<='2010-12-31')]
# df_2011 = df[(df['collision_date']>='2011-01-01') & (df['collision_date']<='2011-12-31')]
# df_2012 = df[(df['collision_date']>='2012-01-01') & (df['collision_date']<='2012-12-31')]
# df_2013 = df[(df['collision_date']>='2013-01-01') & (df['collision_date']<='2013-12-31')]
# df_2014 = df[(df['collision_date']>='2014-01-01') & (df['collision_date']<='2014-12-31')]
# df_2015 = df[(df['collision_date']>='2015-01-01') & (df['collision_date']<='2015-12-31')]
# df_2016 = df[(df['collision_date']>='2016-01-01') & (df['collision_date']<='2016-12-31')]
# df_2017 = df[(df['collision_date']>='2017-01-01') & (df['collision_date']<='2017-12-31')]
# df_2018 = df[(df['collision_date']>='2018-01-01') & (df['collision_date']<='2018-12-31')] 
# print("2009",df_2009.shape)
# print("2010",df_2010.shape)
# print("2011",df_2011.shape)
# print("2012",df_2012.shape)
# print("2013",df_2013.shape)
# print("2014",df_2014.shape)
# print("2015",df_2015.shape)
# print("2016",df_2016.shape)
# print("2017",df_2017.shape)
# print("2018",df_2018.shape)

## dict_2009_to_2018_suburbstats for collision dataset

In [188]:
temp=[]
for df_temp in [df_2009,df_2010,df_2011,df_2012,df_2013,df_2014,df_2015,df_2016,df_2017,df_2018]:
    temp.append(Counter(df_temp['suburb']))
dict_2009_to_2018_suburbstats={}
dict_2009_to_2018_suburbstats[2009] = temp[0]
dict_2009_to_2018_suburbstats[2010] = temp[1]
dict_2009_to_2018_suburbstats[2011] = temp[2]
dict_2009_to_2018_suburbstats[2012] = temp[3]
dict_2009_to_2018_suburbstats[2013] = temp[4]
dict_2009_to_2018_suburbstats[2014] = temp[5]
dict_2009_to_2018_suburbstats[2015] = temp[6]
dict_2009_to_2018_suburbstats[2016] = temp[7]
dict_2009_to_2018_suburbstats[2017] = temp[8]
dict_2009_to_2018_suburbstats[2018] = temp[9]

Sanity check

In [144]:
assert(df.shape[0]==df_2009.shape[0]+df_2010.shape[0]+df_2011.shape[0]+df_2012.shape[0]+df_2013.shape[0]+df_2014.shape[0]+df_2015.shape[0]+df_2016.shape[0]+df_2017.shape[0]+df_2018.shape[0])

In [145]:
for df in [df_2009,df_2010,df_2011,df_2012,df_2013,df_2014,df_2015,df_2016,df_2017,df_2018]:
    lat_mean = np.mean(df['latitude'])
    long_mean = np.mean(df['longitude'])
    lat_var = np.var(df['latitude'])
    long_var =np.var(df['longitude'])
    #myplot = sns.jointplot(x="latitude", y="longitude",kind='kde',data=df)
    print("latitude&longitude variance: ",lat_var+long_var,"latitude&longitude mean: ",lat_mean+long_mean)

latitude&longitude variance:  1.1061358774895167 latitude&longitude mean:  107.22681998152021
latitude&longitude variance:  1.0247501005510875 latitude&longitude mean:  107.22815776117113
latitude&longitude variance:  1.1062828145405939 latitude&longitude mean:  107.24382442860065
latitude&longitude variance:  1.152894392947203 latitude&longitude mean:  107.26057976045493
latitude&longitude variance:  1.4036276223222104 latitude&longitude mean:  107.24225685236495
latitude&longitude variance:  1.2288850870355692 latitude&longitude mean:  107.27547006029002
latitude&longitude variance:  1.1714388334519232 latitude&longitude mean:  107.25159346566105
latitude&longitude variance:  1.2932716364497745 latitude&longitude mean:  107.29019244193438
latitude&longitude variance:  1.2835885956054263 latitude&longitude mean:  107.31134871682339
latitude&longitude variance:  1.2807330681454454 latitude&longitude mean:  107.28753808604324


In [146]:
myplot

filtering out records from 2014 to 2019

In [ ]:
df_2014_to_2019 = my_data_df_1[(my_data_df_1['collision_date']>'2014-01-01') & (my_data_df_1['collision_date']<'2019-01-01')]

In [ ]:
numerical_fields = df_2014_to_2019._get_numeric_data().columns
categorical_fields = list(set(df_2014_to_2019.columns)-set(numerical_fields))
assert len(categorical_fields)+len(numerical_fields)==len(df_2014_to_2019.columns)
len(numerical_fields),len(categorical_fields)

In [ ]:
df_2014_to_2019_cleaned = df_2014_to_2019[['alcohol_involvement_flag','collision_date', 'collision_time','latitude', 'longitude','lga_name', 'country_metro', 'collision_injury_severity','speed_zone']]

In [ ]:
df_2014_to_2019_cleaned.drop(df_2014_to_2019_cleaned[df_2014_to_2019_cleaned.latitude =='.'].index, inplace=True)

df_2014_to_2019_cleaned['latitude']= pd.to_numeric(df_2014_to_2019_cleaned['latitude'])  
df_2014_to_2019_cleaned['longitude']= pd.to_numeric(df_2014_to_2019_cleaned['longitude'])  


In [ ]:
df_2014_to_2019_cleaned.isnull().sum()/len(df_2014_to_2019_cleaned)

In [ ]:
df_2014_to_2019_cleaned.to_csv(working_dir+'/2014_to_2019_Y_N.csv')

In [ ]:
df_Y_N = df_2014_to_2019_cleaned

In [ ]:
lganames = list(pd.Categorical(df_Y_N['lga_name']).categories)

These lga's do not have any accidents due to alcohol in last 5 years

In [ ]:
for i in lganames:
    if "(U" in i:
        lganames.remove(i)
lganames.remove('COLAC OTWAY')
len(lganames)

In [ ]:
df=[]
for lga in lganames:
    temp_df=df_Y_N[df_Y_N['lga_name']==lga]
    df.append(temp_df)

In [ ]:
for i in range(78):
    diff=sum(df[i]['alcohol_involvement_flag']=='N')-sum(df[i]['alcohol_involvement_flag']=='Y')
    if diff==0:
        pass
    elif diff>0:
        [df[i]['alcohol_involvement_flag']=='N'

In [ ]:
import numpy as np
import numpy.random
import scipy.stats as ss
import matplotlib.pyplot as plt

# Set-up.
n = 10000
numpy.random.seed(0x5eed)
# Parameters of the mixture components
norm_params = np.array([[5, 1],
                        [1, 1.3]
                       ])
n_components = norm_params.shape[0]
print(n_components)
# Weight of each component, in this case all of them are 1/3
weights = np.ones(n_components, dtype=np.float64) / 2.0
print(weights)
# A stream of indices from which to choose the component
mixture_idx = numpy.random.choice(len(weights), size=n, replace=True, p=weights)
# y is the mixture sample
y = numpy.fromiter((ss.norm.rvs(*(norm_params[i])) for i in mixture_idx),
                   dtype=np.float64)

# Theoretical PDF plotting -- generate the x and y plotting positions
xs = np.linspace(y.min(), y.max(), 200)
ys = np.zeros_like(xs)

for (l, s), w in zip(norm_params, weights):
    ys += ss.norm.pdf(xs, loc=l, scale=s) * w

plt.plot(xs, ys)
plt.hist(y, normed=True, bins="fd")
plt.xlabel("x")
plt.ylabel("f(x)")
plt.show()

In [ ]:
a=pd.Categorical(my_data['lga_name'])
b=pd.Categorical(df_2014_to_2019_cleaned['lga_name'])
improved_lga = set(b)-set(a)

In [ ]:
b

In [ ]:
improved_lgas_list=[]
for x in improved_lga:
    if my_data['lga_name']==x:
improved_lgas_list.append(my_data[my_data['lga_name']==x])

In [ ]:
# sns.pairplot(alcoholic_crashes_df)
sns.pairplot(df_2014_to_2019_cleaned, vars = list(numerical_fields), hue = 'lga_name', diag_kind = 'kde', plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},size = 4)


In [ ]:
group_by_LGA_2014_to_2019 = df_2014_to_2019_cleaned.groupby(by=['lga_name','latitude','longitude'])
no_alcohol_crashes_by_LGA_2014_to_2019 =  group_by_LGA_2014_to_2019.size().sort_values(ascending=False)#77
frame=pd.DataFrame(no_alcohol_crashes_by_LGA_2014_to_2019)


In [ ]:
no_alcohol_crashes_by_LGA_2014_to_2019.to_csv('lga.csv')

## Fatal crashes in last 5 years

https://vicroadsopendata-vicroadsmaps.opendata.arcgis.com/datasets/fatal-crashes-lives-lost-last-5-years-to-date/data?orderBy=ACCIDENT_DATE&orderByAsc=false

<b>Interactive dashboard</b>
https://public.tableau.com/views/CrashstatMainlandingpage/Mainpage?:embed=y&:display_count=yes&:showTabs=y&:showVizHome=no&%3Adisplay_count=yes&%3Atoolbar=no&%3Arender=false#1

In [ ]:
df_fatal = pd.read_csv(working_dir+'/Fatal_Crashes__Lives_Lost_Last_5_Years_to_Date.csv')

In [ ]:
df_fatal.drop(['X','Y'],axis=1,inplace=True)

In [ ]:
sns.pairplot(df_fatal_2014_to_2019_age, hue = 'street_type', diag_kind = 'kde', plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},size = 4)


In [ ]:
df_joined = df_2014_to_2019.set_index('tis_number').join(alcoholic_crashes_df.set_index('ACCIDENT_NO'))

In [ ]:

df_joined=df_joined[['collision_date', 'collision_time', 'latitude', 'longitude', 'lga_name',
       'country_metro', 'collision_injury_severity', 'speed_zone','DRIVER','YOUNG_DRIVER','OLD_DRIVER', 'UNLICENCSED']]

# EBT_DATA

In [43]:
ebt_df = pd.read_csv(working_dir+'/EBT_Aggregate_data.csv')
print("data shape BEFORE filtering years:",ebt_df.shape)
ebt_df_2014_to_2019 = ebt_df[(ebt_df['intercept year']>=2014) & (ebt_df['intercept year']<2019)]
print("data shape AFTER filtering years:",ebt_df_2014_to_2019.shape)

data shape BEFORE filtering years: (152048, 32)
data shape AFTER filtering years: (47341, 32)


# Aggregate collision data

In [215]:
aggre_df = pd.read_csv(working_dir+'/Aggregate_data.csv')
print("data shape BEFORE filtering years:",aggre_df.shape)
aggre_df_2014_to_2019 = aggre_df[(aggre_df['collision_year']>=2014) & (aggre_df['collision_year']<2019)]
print("data shape AFTER filtering years:",aggre_df_2014_to_2019.shape)

data shape BEFORE filtering years: (501886, 14)
data shape AFTER filtering years: (198249, 14)


In [216]:
aggre_2009 = aggre_df[aggre_df['collision_year']==2009]
aggre_2010 = aggre_df[aggre_df['collision_year']==2010]
aggre_2011 = aggre_df[aggre_df['collision_year']==2011]
aggre_2012 = aggre_df[aggre_df['collision_year']==2012]
aggre_2013 = aggre_df[aggre_df['collision_year']==2013]
aggre_2014 = aggre_df[aggre_df['collision_year']==2014]
aggre_2015 = aggre_df[aggre_df['collision_year']==2015]
aggre_2016 = aggre_df[aggre_df['collision_year']==2016]
aggre_2017 = aggre_df[aggre_df['collision_year']==2017]
aggre_2018 = aggre_df[aggre_df['collision_year']==2018]

In [204]:
temp=[]
for i in [aggre_2009,aggre_2010,aggre_2011,aggre_2012,aggre_2013,aggre_2014,aggre_2015,aggre_2016,aggre_2017,aggre_2018]:
    temp.append(Counter(i['collision_suburb']))
dict_2009_to_2018_aggre_suburbstats={}
dict_2009_to_2018_aggre_suburbstats[2009] = temp[0]
dict_2009_to_2018_aggre_suburbstats[2010] = temp[1]
dict_2009_to_2018_aggre_suburbstats[2011] = temp[2]
dict_2009_to_2018_aggre_suburbstats[2012] = temp[3]
dict_2009_to_2018_aggre_suburbstats[2013] = temp[4]
dict_2009_to_2018_aggre_suburbstats[2014] = temp[5]
dict_2009_to_2018_aggre_suburbstats[2015] = temp[6]
dict_2009_to_2018_aggre_suburbstats[2016] = temp[7]
dict_2009_to_2018_aggre_suburbstats[2017] = temp[8]
dict_2009_to_2018_aggre_suburbstats[2018] = temp[9]

In [274]:
ebt_df.columns

Index(['person age group', 'person gender', 'intercept type', 'collision',
       'intercept suburb', 'intercept postcode', 'intercept lga',
       'LGA Description', 'intercept year', 'intercept month', 'intercept day',
       'Intercept Time', 'place of last drink', 'licenced premises suburb',
       'licenced premises postcode', 'distance prior to intercept',
       'person bac level group', 'full beer', 'light beer', 'wine', 'spirits',
       'cider', 'fortified wine', 'other alcohol', 'alcohol type unknown',
       'person occupation', 'person address suburb',
       'person address post code', 'licence type', 'vehicle type',
       'offence type', 'no of persons'],
      dtype='object')

In [270]:
ebt_df_alcohol['person address suburb']

0                      Corio
2                  Wendouree
4                  Moorabbin
26                Mornington
29        Narre Warren North
33                    Ararat
36               Miners Rest
38                   Mulwala
48                    Albury
50                   Highton
54               Warrnambool
61                       Rye
64                Shepparton
65                 Reservoir
70                  Hastings
96                 Alfredton
100                Bellfield
101                  Mildura
105                     Sale
106               Sebastopol
114                  Wodonga
118              Coronet Bay
129                  Chelsea
130           Altona Meadows
132               Bairnsdale
146           Box Hill North
164                 Cardinia
170                  Bendigo
174                   Echuca
182                St Albans
                 ...        
152003           Keilor East
152004        Doncaster East
152005            Cheltenham
152006        

In [264]:
#ONLY CONSIDERING "ALCOHOL" RELATED CASES FROM EBT DATASET, (DUI/DWI debatable)
ebt_df_alcohol=ebt_df[(ebt_df['offence type'] == "'Z' Condition (0.00)")|(ebt_df['offence type']=='DUI')|(ebt_df['offence type']=='DWI')|(ebt_df['offence type']=='PCA (0.05 / 0.00)')]

The legal blood alcohol limits in Victoria are -

Over the age of 21 - 0.05

Under the age of 21 - 0.00  
<b>PCA:Prescribed Content of Alcohol  
Driving under the influence (DUI)  
driving while impaired/driving while intoxicated (DWI)  
Z condition:zero alcohol levels i.e. under 21 drivers</b>

In [291]:
a=ebt_df['full beer']+ebt_df['light beer'] + ebt_df['wine']+ ebt_df['spirits']+ebt_df['cider']+ebt_df['fortified wine'] +ebt_df['other alcohol']

In [305]:
ebt_df['collision']==1 and ebt_df['intercept type']=='RBT'

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [92]:
# ebt_df_2014_to_2019.isnull().sum()/len(ebt_df_2014_to_2019)
# ebt_df[['intercept suburb', 'intercept postcode', 'intercept lga',
#        'intercept year', 'intercept month', 'intercept day',
#        'Intercept Time']]
sum(ebt_df['intercept year']==2017)

11622

In [55]:
df_2009.head()

,tis_number,collision_date,collision_time,latitude,longitude,street_number,street_name,street_type,suburb,postcode,...,collision_injury_severity,collision_type_code,collision_type,dca_code,dca_code_description,speed_zone,motorcyclist,seat_belt_not_worn,helmet_not_worn,heavy_vehicle
1,T20090000002,2009-01-01,5,-37.884967,145.006103,.,GLEN HUNTLY,RD,ELSTERNWICK,3185.0,...,Injury,4.0,Collision with Fixed Object,181,Off Path on Curve - off Right Bend into Object...,60.0,NaN,NaN,NaN,NaN
6,T20090000010,2009-01-01,245,-37.770396,144.795132,.,WESTERN,HWY,DEER PARK,3023.0,...,Non-Injury,1.0,Collision with Vehicle,110,Vehicles / Adjacent Dirs (Inters) - Cross Traffic,80.0,NaN,NaN,NaN,NaN
10,T20090000016,2009-01-01,305,-36.794980,144.336200,.,ALEXANDER,CL,STRATHFIELDSAYE,3551.0,...,Injury,1.0,Collision with Vehicle,160,On Path – Parked,50.0,NaN,NaN,NaN,NaN
12,T20090000018,2009-01-01,245,-37.783066,145.165642,22,BELLEVUE,AVE,DONCASTER EAST,3109.0,...,Non-Injury,4.0,Collision with Fixed Object,171,Off Path on Straight - Left off Carriageway in...,50.0,NaN,NaN,NaN,NaN
17,T20090000026,2009-01-01,720,-37.961952,145.018981,.,BLUFF,RD,BLACK ROCK,3193.0,...,Injury,4.0,Collision with Fixed Object,172,Off Path on Straight - off Carriageway to Right,60.0,NaN,NaN,NaN,NaN


### Imputing 'licenced premises postcode' with mode i.e. 3000

In [ ]:
df.drop(df[df['intercept postcode']=='.'].index, inplace=True)
df['intercept postcode']=pd.to_numeric(df['intercept postcode'])

In [ ]:
df=df

In [ ]:
numerical_fields

In [ ]:
plot_scatter = False
numerical_fields =df._get_numeric_data().columns
print(numerical_fields)
categorical_fields = list(set(df.columns)-set(numerical_fields))
assert len(categorical_fields)+len(numerical_fields)==len(df.columns)
for i in numerical_fields:
    for j in numerical_fields:
        if i==j:
            sns.distplot(df[i])
            plt.show()
        elif i!=j and plot_scatter:
            sns.scatterplot(x=i,y=j,data=df)
            plt.show()

In [ ]:
for i,lastplace in enumerate(pd.Categorical(df['place of last drink']).categories):
    print(i+1,lastplace)

In [ ]:
categorical_fields

In [ ]:
#sns.pairplot(df, vars = list(numerical_fields), hue =   'offence type', diag_kind = 'kde', plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},size = 4)
pd.Categorical(df['offence type']).describe()

## collision data

In [ ]:
aggre_df_2014_to_2019.columns

In [ ]:
df = aggre_df_2014_to_2019

In [ ]:
df.isnull().sum()/len(df)

In [ ]:
pd.Categorical(df['person_bac_level_group']).describe()

In [ ]:
plot_scatter = False
numerical_fields = df._get_numeric_data().columns
print(numerical_fields)
categorical_fields = list(set(df.columns)-set(numerical_fields))
assert len(categorical_fields)+len(numerical_fields)==len(df.columns)
for i in numerical_fields:
    for j in numerical_fields:
        if i==j:
            sns.distplot(df[i])
            plt.show()
        elif i!=j and plot_scatter:
            sns.scatterplot(x=i,y=j,data=df)
            plt.show()